# Volume-Based Estimators

The idea of estimators... **citations**

When TARDIS runs, we enter into a loop with two main components: a Monte Carlo iteration occurs, and then the plasma state is updated based on the estimators.

## Theory

To account for the effects of the Monte Carlo packets on the ejecta, TARDIS uses the packets to first determine the average energy density $E$ throughout each shell. To do this, for each shell we sum up the amount of energy each packet has ($\epsilon$) multiplied by how much time it spends in the shell ($\delta t$) relative to our reference time of the simulation ($\Delta t$; this is calculated in [Energy Packet Initialization](../montecarlo/initialization.ipynb)), and then divide by the volume of the shell ($\Delta V$). That is,

$$E=\frac{1}{\Delta V}\sum_i \epsilon_i \frac{\delta t_i}{\Delta t}$$

where we sum over every Monte Carlo packet in the shell (in the implimentation section, we will explain how to account for the fact that the energy of the packet can change throughout the simulation ????????????).

Using this energy density, we can 

## Implimentation

**Mention that each iteration is just running the model and storing the state, plus some other stuff for recording things etc**

## Line Estimators

section

## Code Example

We now show a detailed example of how the plasma is updated after a Monte Carlo iteration using the estimators. First, we import the needed packages and set up a simulation (see **link**):

In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
import numpy as np

/home/smithis7/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/smithis7/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each radial cell (see **link**):

In [3]:
model.t_rad

<Quantity [9926.50196546, 9911.63537753, 9896.81325339, 9882.03539385,
           9867.30160093, 9852.6116778 , 9837.96542884, 9823.36265956,
           9808.80317663, 9794.28678787, 9779.81330223, 9765.3825298 ,
           9750.99428177, 9736.64837045, 9722.34460927, 9708.08281272,
           9693.8627964 , 9679.68437699, 9665.54737223, 9651.45160094] K>

In [4]:
model.w

array([0.40039164, 0.33245223, 0.28966798, 0.2577141 , 0.23224568,
       0.21120466, 0.19341188, 0.17811402, 0.16479446, 0.1530809 ,
       0.14269498, 0.13342262, 0.12509541, 0.11757849, 0.11076215,
       0.10455605, 0.09888494, 0.09368554, 0.08890421, 0.08449514])

In [5]:
plasma.get_value('electron_densities')

0     2.865134e+09
1     2.182365e+09
2     1.678840e+09
3     1.303472e+09
4     1.020811e+09
5     8.059447e+08
6     6.411609e+08
7     5.137297e+08
8     4.144079e+08
9     3.364195e+08
10    2.747519e+08
11    2.256656e+08
12    1.863476e+08
13    1.546660e+08
14    1.289928e+08
15    1.080764e+08
16    9.094799e+07
17    7.685317e+07
18    6.520063e+07
19    5.552442e+07
dtype: float64

In [6]:
plasma.get_value('tau_sobolevs')

0   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 6.110053e-12   
                         177                191                 4.581877e-12   
                         178                191                 3.048177e-13   
                         138                144                 2.205327e-10   
                         177                190                 8.129552e-14   

                                                                          1   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 2.765775e-12   
                         177                191                 2.074031e-12   
                         178                191                 1.379787e-13   
                         138                144                 9.983538e-11   
                         177                190                 3.679921e-14   

                                                                          2   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 1.290338e-12   
                         177                191                 9.676133e-13   
                         178                191                 6.437225e-14   
                         138                144                 4.658121e-11   
                         177                190                 1.716821e-14   

                                                                          3   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 6.191522e-13   
                         177                191                 4.642969e-13   
                         178                191                 3.088820e-14   
                         138                144                 2.23534

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [31]:
N_packets = 10000

# Include getting n_packets from sim?

sim.iterate(N_packets)

We now show the values of the three estimators previously mentioned:

In [8]:
runner.j_estimator

array([1.13264880e+14, 9.80619708e+13, 8.77127720e+13, 7.93178023e+13,
       7.33452674e+13, 6.69288783e+13, 6.42825236e+13, 6.09589567e+13,
       5.76244911e+13, 5.49744149e+13, 5.27742170e+13, 5.11516006e+13,
       4.94278177e+13, 4.78690453e+13, 4.66549201e+13, 4.55199706e+13,
       4.46151244e+13, 4.38293494e+13, 4.31417403e+13, 4.22137375e+13])

In [9]:
runner.nu_bar_estimator

array([9.23669721e+28, 8.00440961e+28, 7.16701149e+28, 6.52233231e+28,
       6.04286574e+28, 5.48759864e+28, 5.29893354e+28, 5.02177670e+28,
       4.72808091e+28, 4.50654639e+28, 4.30918375e+28, 4.12731776e+28,
       3.97685768e+28, 3.81488727e+28, 3.68151763e+28, 3.55929590e+28,
       3.46700820e+28, 3.38472442e+28, 3.32955181e+28, 3.24241018e+28])

Show t_rad and w calculations??

In [10]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

array([[2.54208693e-19, 3.98921452e-19, 3.51022121e-19, ...,
        4.78256257e-20, 1.83610133e-19, 4.62693098e-20],
       [5.25249079e-20, 3.47850697e-19, 3.53175242e-19, ...,
        4.68369084e-20, 4.76896351e-20, 1.42425885e-19],
       [5.25205919e-20, 3.47822114e-19, 3.03857848e-19, ...,
        4.68330598e-20, 4.76857164e-20, 1.42414182e-19],
       ...,
       [9.16402388e-19, 4.44096419e-19, 8.75723476e-19, ...,
        1.23926509e-18, 8.25398566e-19, 8.28841475e-19],
       [9.16135899e-19, 4.43967276e-19, 4.26276968e-19, ...,
        8.28490409e-19, 1.23557284e-18, 8.28600449e-19],
       [2.29780963e-18, 9.13630461e-19, 2.30523331e-18, ...,
        4.23437186e-19, 4.28206752e-19, 4.22703828e-19]])

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [11]:
plasma.get_value('tau_sobolevs').shape, runner.j_blue_estimator.shape

((29224, 20), (29224, 20))

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [32]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

Shell No.,t_rad,next_t_rad,w,next_w
0,1.11e+04,1.12e+04,0.468,0.452
5,1.15e+04,1.13e+04,0.167,0.18
10,1.13e+04,1.13e+04,0.106,0.104
15,1.09e+04,1.1e+04,0.0792,0.0773


In [13]:
model.t_rad

<Quantity [10212.74995587, 10222.33124932, 10232.84808982, 10298.01481758,
           10317.91893208, 10268.09920614, 10323.25883794, 10316.70789091,
           10275.40760376, 10266.07641642, 10225.73367215, 10104.851613  ,
           10076.03983165,  9980.40591063,  9882.13306024,  9792.27010212,
            9731.81900046,  9671.18192097,  9665.16737459,  9619.12152881] K>

In [14]:
model.w

array([0.48154582, 0.3839523 , 0.31710814, 0.25991916, 0.22230488,
       0.19324106, 0.17011455, 0.15176297, 0.13706675, 0.12362115,
       0.11375669, 0.10928659, 0.10111279, 0.09644048, 0.09283222,
       0.0893001 , 0.08539126, 0.08195894, 0.0771517 , 0.07348597])

In [15]:
plasma.get_value('electron_densities')

0     2.890015e+09
1     2.199909e+09
2     1.691446e+09
3     1.313845e+09
4     1.028790e+09
5     8.114482e+08
6     6.459942e+08
7     5.175646e+08
8     4.172918e+08
9     3.387728e+08
10    2.765659e+08
11    2.267949e+08
12    1.872454e+08
13    1.552250e+08
14    1.292984e+08
15    1.082114e+08
16    9.099949e+07
17    7.684337e+07
18    6.520026e+07
19    5.549692e+07
dtype: float64

In [16]:
plasma.get_value('tau_sobolevs')

0   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 3.751288e-12   
                         177                191                 2.813059e-12   
                         178                191                 1.871440e-13   
                         138                144                 1.351661e-10   
                         177                190                 4.991160e-14   

                                                                          1   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 1.623835e-12   
                         177                191                 1.217700e-12   
                         178                191                 8.100975e-14   
                         138                144                 5.850663e-11   
                         177                190                 2.160543e-14   

                                                                          2   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 7.234954e-13   
                         177                191                 5.425431e-13   
                         178                191                 3.609368e-14   
                         138                144                 2.606589e-11   
                         177                190                 9.626242e-15   

                                                                          3   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 3.026676e-13   
                         177                191                 2.269679e-13   
                         178                191                 1.509946e-14   
                         138                144                 1.09003